In [1]:
pwd

'C:\\Users\\go121\\practice_jupyter\\py\\20241011'

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv('./전세사기 세부내용_네이버지식인_개인.csv')
q_list = df['질문'].unique()

In [4]:
text_res = []
for i in q_list:
    text_res.extend(i.split(' '))
okt = Okt()
phr_word = [okt.phrases(i) for i in text_res]
key_word = sorted(list(set([j for i in phr_word for j in i])))

In [5]:
# 1. 각 문서별 명사, 형용사, 동사 추출[['ㅇㄹ','ㅇㄹ']]
text_res = []
okt = Okt()
for i in q_list:
    text_res.append(okt.nouns(i))

In [6]:
# 2. 단어리스트 : 모든 문서에 사용된 단어들 (컬럼)
key_word = sorted(list(set([j for i in text_res for j in i])))

In [7]:
# 3. tf -> DTM 만들기
dtm = []
for i in key_word:
    dtm.append([])
    for j in text_res:
        dtm[-1].append(j.count(i))

In [8]:
tf = pd.DataFrame(dtm,index=key_word).T
tf

,가가,가게,가격,가구,가능,가능성,가도,가량,가압류,가야,...,환,환불,환전,환청,회사,회생,회수,효력,후,후회
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,4,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df = tf.astype('bool').sum()
df

가가     1
가게     1
가격     1
가구     1
가능     4
      ..
회생    10
회수     2
효력     2
후     16
후회     1
Length: 1015, dtype: int64

In [10]:
D = len(tf)
D

100

In [11]:
# 4. df -> idf 구하기 log이용 -> tf_idf 구하기
idf = D/df
idf

가가    100.00
가게    100.00
가격    100.00
가구    100.00
가능     25.00
       ...  
회생     10.00
회수     50.00
효력     50.00
후       6.25
후회    100.00
Length: 1015, dtype: float64

In [12]:
tfidf_ = tf*idf
tfidf_

,가가,가게,가격,가구,가능,가능성,가도,가량,가압류,가야,...,환,환불,환전,환청,회사,회생,회수,효력,후,후회
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,33.333333,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# 5. 문서유사도 : 첫 번째 문서와 가장 유사한 문서는 몇 번째인지?
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(q_list)
tfidf_vectorizer

TfidfVectorizer()

In [14]:
tfidf_feature = tfidf_vectorizer.get_feature_names_out()
tfidf_feature

array(['01', '02일에', '05에', ..., '힘들까요', '힘들어서', '힘들어할'], dtype=object)

In [15]:
tfidf_matrix.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.11163301, 0.11163301, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature)

,01,02일에,05에,09,0살하고,10,1000만원,100채,100프로,10개월,...,효력이,후순위,후순위거나,후에,후회막심입니다,힘드내요,힘들거,힘들까요,힘들어서,힘들어할
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.111633,0.111633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
